In [1]:
%%time
import warnings
warnings.filterwarnings('ignore') # to ignore annoying IPython warnings
import numpy as np
import pandas as pd
import gzip
import re

from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix, hstack # to get memory-efficient representation of matrices (sparse format)
from textblob import TextBlob, Word

# preprocessing / feature extraction / feature transformation
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import PCA, SparsePCA

# models
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline, FeatureUnion

# metrics/validation
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
# model serialization/deserialization
import dill

Wall time: 52.4 s


In [1]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import tree
import pandas as pd
import pydotplus
import dill

True

In [16]:
train_df_names = ['reviews_rt_all.csv', 'imdb_small.csv']
train = pd.concat((pd.read_csv(name, engine='c', sep='|', 
                 usecols=['label', 'text']) for name in train_df_names), ignore_index=True)
print('review count: {}'.format(len(train)))

# check for class balance
print('class balance:', '\n', train.label.value_counts())

review count: 152610
class balance: 
 1    89658
0    62952
Name: label, dtype: int64


In [38]:
df = pd.concat([train[train.label == 1].sample(50, random_state=48),  train[train.label == 0].sample(50, random_state=48)])

In [39]:
df.head(50)

,label,text
32866,1,This superbly composed film comes as close to ...
69388,1,A solid character piece that deftly probes the...
32178,1,More a curio than anything else.
96892,1,Traz o peso de tragédia grega que Woody Allen ...
80587,1,"Met on its own bonkers terms, Prince Of Darkne..."
72548,1,"Cold, masterly, without pathos, and not even p..."
32970,1,One of the best movie versions of Batman ever
59138,1,Nothing particularly exciting or boring about ...
70094,1,"Unfussily directed, To Have And Have Not is pa..."
91069,1,"an eccentric, audacious musical that, like ""Mo..."


In [43]:
extraction_list = []
# 2. simple bag-of-words (tf-idf)
extraction_list.append(['tfidf', 
                             TfidfVectorizer(decode_error='ignore',
                                             max_df=0.5, 
                                             min_df=3,
                                             ngram_range=(1, 3),
                                             max_features=None,
                                             stop_words='english'
                                            )
                            ])

extractor = FeatureUnion(extraction_list)

clf = tree.DecisionTreeClassifier(min_samples_leaf=3,
                            max_leaf_nodes=3,
                            max_depth=4)

# create pipeline, combining steps together                                                                                                                       
model = Pipeline(
    [
        ('feature_extraction', extractor),
        ('clf', clf)
    ])

In [44]:

clf = model.fit(df.text, df.label)

#model.fit(df.text, df.label)
print(clf, 'finally fitted :)')

Pipeline(steps=[('feature_extraction', FeatureUnion(n_jobs=1,
       transformer_list=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='ignore',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=3,
     ...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))]) finally fitted :)


In [45]:
text = pd.Series(['I dont like it', 'nice film!', 'it is total crap. 2/10',
                  'This show is a must have if you enjoy shows with family Michael J fox does a spectacular job playing Alex Keaton and the series finale is great'])
                        
pred = clf.predict(text)
print(pred)

[0 0 0 0]


In [31]:
dot_data = tree.export_graphviz(clf.named_steps['clf'], out_file=None)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_png('tree1.png')

AttributeError: 'ExtraTreesClassifier' object has no attribute 'tree_'